In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression, PoissonRegressor
from sklearn.metrics import r2_score

from spineq.data_fetcher import get_oa_shapes, get_oa_centroids, get_la_shape
from spineq.plotting import plot_oa_weights, get_color_axis, plot_optimisation_result
from spineq.greedy import greedy_opt
from spineq.optimise import make_result_dict, optimise
from spineq.utils import coverage_matrix

import contextily as ctx
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("talk")
%matplotlib inline


In [ ]:
lad20cd = "E08000037" # "E08000021"  # "E08000037" 

## Space Syntax

In [ ]:
ss = gpd.read_file("../data/raw/space_syntax/TyneandWear_geojson.geojson")
ss.rename(columns={"id": "ss_id"}, inplace=True)

In [ ]:
# only keep space syntax segments intersecting or within local authority geometry
la = get_la_shape(lad20cd)

ss = ss[
    ss.crosses(la.geometry)
    | ss.within(la.geometry)
]

In [ ]:
ax = plt.figure(figsize=(20, 20)).gca()
cax = get_color_axis(ax)
ax = ss.plot(
    column="AC__NACH", alpha=1, cmap="YlOrRd", legend=True, ax=ax, cax=cax, linewidth=4, zorder=2
)

ctx.add_basemap(
    ax,
    source="http://a.tile.stamen.com/toner/{z}/{x}/{y}.png",
    crs=ss.crs.to_epsg(),
)
ax.set_title("AC__NACH")
ax.set_axis_off()
gpd.GeoSeries(la.geometry, crs=ss.crs).plot(
    ax=ax, facecolor="None", edgecolor="b", linewidth=5, zorder=1
)

In [ ]:
ss["AC__NACH"].plot.hist(bins=20)

## Raw DFT Traffic Data

In [ ]:
dft = pd.read_csv("../data/raw/dft/dft_traffic_counts_aadf.csv")
display(dft.head())
len(dft)

In [ ]:
select_year = 2019
dft = dft[dft["year"] == select_year]
display(dft.head())
len(dft)

In [ ]:
dft = gpd.GeoDataFrame(dft, geometry=gpd.points_from_xy(dft["easting"], dft["northing"]),  crs="epsg:27700")
dft.rename(columns={"id": "dft_id"}, inplace=True)

In [ ]:
dft = dft[
    dft.crosses(la.geometry)
    | dft.within(la.geometry)
]

display(dft)
len(dft)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 20))
ss.plot(ax=ax, zorder=1, color="gray", linewidth=0.75)
dft.plot(ax=ax, markersize=64, column=dft["road_name"].str[0], zorder=2, legend=True)
ax.axis("off")
ax.set_title("DfT Traffic Count Points", fontsize=20)

### Snap DFT points to nearest  lines (roads) in space syntax model

Following approach from:
- https://medium.com/@brendan_ward/how-to-leverage-geopandas-for-faster-snapping-of-points-to-lines-6113c94e59aa


In [ ]:
def match_dft_ss(
    dft: pd.DataFrame, ss: gpd.GeoDataFrame, offset: float = 10, tolerance: float = 50
) -> gpd.GeoDataFrame:
    """
    Find the line segments in the space syntax data ss that are closest to the
    traffic measurement points in dft.

    Inspired by https://medium.com/@brendan_ward/6113c94e59aa
    """
    ss.sindex  # Create spatial index (makes computation faster later)

    # all space syntax (ss) segments that intersect DfT measurement points
    # (within offset metres east/north of measurement point):
    bbox = dft.bounds + [-offset, -offset, offset, offset]
    hits = bbox.apply(lambda row: list(ss.sindex.intersection(row)), axis=1)

    # convert hits to flat list (row cotaining list of matches, to rows each
    # containing 1 match)
    hits = pd.DataFrame(
        {
            # index of point in dft
            "pt_idx": np.repeat(hits.index, hits.apply(len)),
            # index of line segment in ss
            "line_idx": np.concatenate(hits.values),
        }
    )

    # Join ss and dft based on matched indices
    hits = hits.join(ss.reset_index(drop=True), on="line_idx")
    hits = hits.join(dft.rename(columns={"geometry": "point"}), on="pt_idx")
    hits = gpd.GeoDataFrame(hits, geometry="geometry", crs=dft.crs)

    # calculate distances between points and matched line segments
    hits["snap_dist"] = hits.geometry.distance(gpd.GeoSeries(hits.point))
    # discard any hits with more than tolerance metres between dft point
    # and ss segment
    hits = hits.loc[hits.snap_dist <= tolerance]

    # Select closest dft point and ss segment match
    hits = hits.sort_values(by=["snap_dist"])
    closest = hits.groupby("pt_idx").first()

    return gpd.GeoDataFrame(closest, geometry="geometry")

In [ ]:
dft_ss = match_dft_ss(dft, ss)
dft_ss.head()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 20))
ss.plot(ax=ax, zorder=1, linewidth=0.75, color="gray")
dft_ss.plot(ax=ax, zorder=2, color="r", linewidth=5)
ax.axis("off")
ax.set_title("Space Syntax Segments with DfT Measurement", fontsize=20)
border = 100
ax.set_xlim([dft["easting"].min() - border, dft["easting"].max() + border])
ax.set_ylim([dft["northing"].min() - border, dft["northing"].max() + border])
dft_ss["point"].plot(ax=ax, markersize=64, zorder=3)


In [ ]:
column = "AC__NACH"

plt.figure(figsize=(10, 5))
sns.scatterplot(
    x=dft_ss[column], y=dft_ss["all_motor_vehicles"], hue=dft_ss["road_name"].str[0]
)

plt.figure(figsize=(10, 5))
sns.scatterplot(
    x=np.log(dft_ss[column]),
    y=np.log(dft_ss["all_motor_vehicles"]),
    hue=dft_ss["road_name"].str[0],
)
plt.xlabel(f"log[{column}]")
plt.ylabel("log[all_motor_vehicles]")

In [ ]:
print(np.corrcoef(np.log(dft_ss["AC__NACH"]), np.log(dft_ss["all_motor_vehicles"])))
spearmanr(dft_ss["AC__NACH"], dft_ss["all_motor_vehicles"])

## Fit

In [ ]:
mask = (dft_ss["AC__NACH"] > 0) & (dft_ss["all_motor_vehicles"] > 0)

X = dft_ss[mask]["AC__NACH"].values
y = dft_ss[mask]["all_motor_vehicles"]

mdl = PoissonRegressor(fit_intercept=True)
mdl.fit(X.reshape(-1, 1), y)
print("Intercept:", mdl.intercept_, ", Coef:", mdl.coef_)
print("D^2:", mdl.score(X.reshape(-1, 1), y))

y_pred = mdl.predict(X.reshape(-1, 1))
resid = y_pred - y
dft_ss.loc[mask, "resid"] = resid
dft_ss.loc[mask, "pred"] = y_pred

plt.figure(figsize=(10, 5))
sns.scatterplot(x=X, y=y, hue=dft_ss[mask]["road_name"].str[0])
x_range = np.linspace(plt.xlim()[0], plt.xlim()[1], 100)
plt.plot(x_range, mdl.predict(x_range.reshape(-1, 1)), "k")
plt.xlabel("AC__NACH")
plt.ylabel("all_motor_vehicles")
plt.title("Fit")

plt.figure(figsize=(10, 5))
sns.scatterplot(
    x=dft_ss.loc[mask, "all_motor_vehicles"],
    y=dft_ss.loc[mask, "pred"],
    hue=dft_ss[mask]["road_name"].str[0],
)
xlim = plt.xlim()
plt.plot(xlim, xlim, "k--")
plt.xlabel("Actual all_motor_vehicles")
plt.ylabel("Predicted all_motor_vehicles")
plt.title("Actuals")

plt.figure(figsize=(10, 5))
sns.scatterplot(
    x=dft_ss.loc[mask, "all_motor_vehicles"],
    y=dft_ss.loc[mask, "pred"] - dft_ss.loc[mask, "all_motor_vehicles"],
    hue=dft_ss[mask]["road_name"].str[0],
)
plt.xlabel("Actual all_motor_vehicles")
plt.ylabel("Actual - Predicted")
plt.title("Residuals")

## Max Space Syntax in Output Area

In [ ]:
oa = get_oa_shapes(lad20cd)
oa

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

test_oa = oa.sample().index[0]
#oa["select"] = False
#oa.loc[test_oa, "select"] = True
#oa.plot(column="select", ax=ax)
#oa.plot(color="None", ax=ax, linewidth=0.5)
ss.plot(ax=ax, linewidth=0.5)
ss[
    ss.crosses(oa.loc[test_oa, "geometry"])
    | ss.within(oa.loc[test_oa, "geometry"])
].plot(color="r", ax=ax)
plt.title(test_oa)

In [ ]:
oa["AC__NACH_max"] = [
    ss[
        ss.crosses(oa.loc[oa_name, "geometry"]) | ss.within(oa.loc[oa_name, "geometry"])
    ]["AC__NACH"].max()
    for oa_name in oa.index
]
oa["AC__NACH_max"].fillna(1e-6, inplace=True)
oa["AC__NACH_max"]

In [ ]:
oa_segments = [
    (ss.crosses(oa.loc[oa_name, "geometry"]) | ss.within(oa.loc[oa_name, "geometry"])).sum()
    for oa_name in oa.index
]

In [ ]:
oa_segments = pd.Series(oa_segments, index=oa.index)
oa_segments.describe()

In [ ]:
# how many space syntax segments in each oa
oa_segments.plot.hist(bins=20)

plt.figure(figsize=(10, 5))
import seaborn as sns
sns.ecdfplot(oa_segments)

In [ ]:
# fix log(NaN) and log(0)
oa["AC__NACH_max"].fillna(1e-6, inplace=True)
oa["AC__NACH_max"].replace(0, 1e-6, inplace=True)

oa["traffic_max"] = mdl.predict(oa["AC__NACH_max"].values.reshape(-1, 1))

oa

In [ ]:
oa["AC__NACH_max"].describe()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
plot_oa_weights(
    lad20cd,
    oa["traffic_max"],
    vmin=0,
    vmax=None,
    ax=ax
)


## Traffic-Optimised Network

In [ ]:
centroids = get_oa_centroids(lad20cd)
oa = oa.join(centroids)

In [ ]:
n_sensors = 50
theta = 2000

In [ ]:
cm =  coverage_matrix(oa["x"], oa["y"], theta=theta)

r = greedy_opt(n_sensors, cm, oa["traffic_max"].values)

r = make_result_dict(
    lad20cd,
    n_sensors,
    theta,
    oa["x"].values,
    oa["y"].values,
    oa.index,
    r["sensors"],
    r["total_coverage"],
    r["point_coverage"],
    list(oa.index[r["placement_history"]]),
    r["coverage_history"],
    oa_weight=r["weights"],
    pop_age_groups={},
    population_weight=0,
    workplace_weight=0,
)

In [ ]:
plot_optimisation_result(r)

## spineq function

In [ ]:
r = optimise(
    lad20cd=lad20cd,
    n_sensors=n_sensors,
    theta=theta,
    population_weight=0,
    workplace_weight=0,
    traffic_weight=1,
)
plot_optimisation_result(r)